In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
im = mnist.train.images[0, :]

In [ ]:
def convolutional(X, height, width, n_input, n_output, activation=True, scope=None):
    # use the input scope or default to "conv"
    with tf.variable_scope(scope or 'conv'):
        W = tf.get_variable(
            name='W',
            shape=[height, width, n_input, n_output],
            initializer=tf.truncated_normal_initializer(stddev=0.1))
        b = tf.get_variable(
            name='b',
            shape=[n_output],
            initializer=tf.constant_initializer())
        h = tf.nn.conv2d(X, W, strides=[1,1,1,1], padding='SAME') + b        
        if activation:
            h = tf.nn.relu(h)
        return h

In [ ]:
def fc(X, n_input, n_output, activation=True, scope=None):
    # use the input scope or default to "conv"
    with tf.variable_scope(scope or 'fc'):
        W = tf.get_variable(
            name='W',
            shape=[n_input, n_output],
            initializer=tf.truncated_normal_initializer(stddev=0.1))
        b = tf.get_variable(
            name='b',
            shape=[n_output],
            initializer=tf.constant_initializer())
        h = tf.reshape(X, [-1, n_input])
        h = tf.matmul(h, W) + b
        if activation:
            h = tf.nn.relu(h)
        return h

In [ ]:
# placeholder for images (in batches)
X = tf.placeholder(tf.float32)
# placeholder for labels (one-hot format)
y_ = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)
X1 = tf.expand_dims(X, 1)
X2 = tf.reshape(X1, [-1, 28, 28, 1])
h1 = convolutional(X2, 5, 5, 1, 32, scope='conv1')
h2 = tf.nn.max_pool(h1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
h3 = convolutional(h2, 5, 5, 32, 64, scope='conv2')
h4 = tf.nn.max_pool(h3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
h5 = fc(h4, 7*7*64, 1024, scope='fc3')
h5_drop = tf.nn.dropout(h5, keep_prob)
y = fc(h5_drop, 1024, 10, scope='fc4')

In [ ]:
# The loss we want to minimize
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))

# The solver
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Performance measure
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Setup the training
n_epochs = 20000
batch_sz = 50
print_err_every = 100

In [ ]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for i in range(n_epochs):
        batch = mnist.train.next_batch(batch_sz)
        if i%print_err_every==0:
            train_accuracy = sess.run(accuracy, feed_dict={X:batch[0], y_: batch[1], keep_prob:1.0})
            print("step %d, training accuracy %g"%(i, train_accuracy))
            
        sess.run(train_step, feed_dict={X: batch[0], y_: batch[1], keep_prob:0.5})